In [2]:
import os
from copy import deepcopy

import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import SplineTransformer,PolynomialFeatures,StandardScaler,MinMaxScaler



from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# MODELS
from sklearn.neighbors import NearestNeighbors

In [3]:
def mean(list):
    return sum(list)/len(list)

def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [4]:
# COLLABORATIVE RECS -- Future application; need users

### Applying weightings to homes that have been upvoted by users with similar profile characteristics

### Strength of weighting should be inversely related to how close user input is to the user input that liked that house
### Potentially use deviations of scaler to determine how far a value is away from norm

# Python --> Crab library

In [5]:
# User Input Considerations:

# I'm Feeling Lucky -> Drop a random dot in the housing map

## Price Range: Min - Max

## Property Type

# Minimum Requirements ^^

## State

## City

## Zip Code

## No. of Bedrooms

## No. of Bathrooms

## Square Footage

## Year Built



### NEED TO IDENTIFY WHEN DATA IS SPARSE FOR A CERTAIN AREA AND PRESENT APOLOGY MESSAGE


In [295]:
aws_bucket=os.getenv('AWS_RE_BUCKET')
aws_key=os.getenv('AWS_RE_BUCKET_KEY')
aws_secret=os.getenv('AWS_RE_BUCKET_SECRET')

date="2023-06-16"
state="CA"
states=['AL',
 'NE',
 'AK',
 'NV',
 'AZ',
 'NH',
 'AR',
 'NJ',
 'CA',
 'NM',
 'CO',
 'NY',
 'CT',
 'NC',
 'DE',
 'ND',
 'DC',
 'OH',
 'FL',
 'OK',
 'GA',
 'OR',
 'HI',
 'PA',
 'ID',
 'PR',
 'IL',
 'RI',
 'IN',
 'SC',
 'IA',
 'SD',
 'KS',
 'TN',
 'KY',
 'TX',
 'LA',
 'UT',
 'ME',
 'VT',
 'MD',
 'VA',
 'MA',
 'VI',
 'MI',
 'WA',
 'MN',
 'WV',
 'MS',
 'WI',
 'MO',
 'WY',
 'MT',
]

In [296]:
# PULLING IN DATA

df_list=[]
for st in states:
    try:
        df = pd.read_parquet(path=aws_bucket+f"/{date}/{st}.parquet",storage_options={"key":aws_key,"secret":aws_secret})
        df_list.append(df)
    except:
        pass
    
df=pd.concat(df_list)

In [9]:
df['PROPERTY TYPE'].unique()

array(['Mobile/Manufactured Home', 'Vacant Land',
       'Single Family Residential', 'Multi-Family (2-4 Unit)',
       'Multi-Family (5+ Unit)', 'Townhouse', 'Ranch', 'Condo/Co-op',
       'Other', 'Parking', 'Timeshare', 'Moorage', 'Unknown', 'None'],
      dtype=object)

In [297]:
df_copy=deepcopy(df)

In [298]:
df_copy.columns

Index(['SALE TYPE', 'TimeStamp', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS',
       'CITY', 'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS',
       'BATHS', 'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT',
       'DAYS ON MARKET', '$/SQUARE FEET', 'HOA/MONTH', 'STATUS',
       'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [299]:
df=df_copy

In [300]:
# PREPROCESSING
prop_types=['Single Family Residential', 'Mobile/Manufactured Home','Townhouse', 'Multi-Family (2-4 Unit)', 'Condo/Co-op','Multi-Family (5+ Unit)']

df=df.applymap(lambda x: np.nan if x=='nan' else x)

#zip_count = df[['ZIP OR POSTAL CODE']].nunique().iloc[0]

#df=df[(df['CITY']=='Houston')&(df['STATE OR PROVINCE']=='TX')] # TEMP LIMITER 1

#df=df[df['PROPERTY TYPE']=='Single Family Residential'] # TEMP LIMITER 2

df=df[df['PROPERTY TYPE'].isin(prop_types)]

df=df[['ADDRESS','CITY','STATE OR PROVINCE','ZIP OR POSTAL CODE','PROPERTY TYPE','PRICE','BEDS','BATHS','SQUARE FEET','YEAR BUILT','URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)']].dropna(axis=0).reset_index(drop=True).apply(lambda row: pd.to_numeric(row,errors='ignore'))


df_full=df

df_final=df_full.drop(['ADDRESS','CITY','STATE OR PROVINCE','ZIP OR POSTAL CODE','PROPERTY TYPE'],axis=1)


In [301]:
df

,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PROPERTY TYPE,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)
0,4900 County Road 15,Maplesville,AL,36750,Mobile/Manufactured Home,179000.0,4.0,2.0,1976.0,2019.0,https://www.redfin.com/AL/Maplesville/4900-Cou...
1,9455 Highway 191,Maplesville,AL,36750,Single Family Residential,650000.0,5.0,3.0,4236.0,2010.0,https://www.redfin.com/AL/Maplesville/9455-AL-...
2,185 County Road 10,Maplesville,AL,36750,Single Family Residential,345000.0,5.0,4.0,2690.0,2002.0,https://www.redfin.com/AL/Maplesville/185-Coun...
3,231 Birmingham St,Prichard,AL,36610,Single Family Residential,154000.0,3.0,2.0,1624.0,1950.0,https://www.redfin.com/AL/Prichard/231-Birming...
4,1417 Saint Claire Ave,Mobile,AL,36610,Multi-Family (2-4 Unit),38000.0,4.0,2.0,1400.0,1950.0,https://www.redfin.com/AL/Mobile/1417-Saint-Cl...
...,...,...,...,...,...,...,...,...,...,...,...
500770,28 Miles Ave Unit 103,Whitefish,MT,59937,Condo/Co-op,2500000.0,3.0,2.5,2707.0,2023.0,https://www.redfin.com/MT/Whitefish/28-Miles-A...
500771,57 Crestwood Dr Unit A,Whitefish,MT,59937,Townhouse,2906000.0,5.0,4.0,2906.0,2022.0,https://www.redfin.com/MT/Whitefish/57-Crestwo...
500772,1515 Highway 93 W Unit 326,Whitefish,MT,59937,Condo/Co-op,1999000.0,3.0,3.0,1960.0,2022.0,https://www.redfin.com/MT/Whitefish/1515-U-S-9...
500773,1515 Highway 93 W Unit 327,Whitefish,MT,59937,Condo/Co-op,1699000.0,2.0,2.0,1698.0,2022.0,https://www.redfin.com/MT/Whitefish/1515-U-S-9...


In [292]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490988 entries, 0 to 490987
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ADDRESS             490988 non-null  object 
 1   CITY                490988 non-null  object 
 2   STATE OR PROVINCE   490988 non-null  object 
 3   ZIP OR POSTAL CODE  490988 non-null  object 
 4   PROPERTY TYPE       490988 non-null  object 
 5   PRICE               490988 non-null  float64
 6   BEDS                490988 non-null  float64
 7   BATHS               490988 non-null  float64
 8   SQUARE FEET         490988 non-null  float64
 9   YEAR BUILT          490988 non-null  float64
 10  DAYS ON MARKET      490988 non-null  float64
 11  LATITUDE            490988 non-null  float64
 12  LONGITUDE           490988 non-null  float64
dtypes: float64(8), object(5)
memory usage: 48.7+ MB


In [209]:
df_NN=df_final[['BEDS','BATHS','SQUARE FEET','YEAR BUILT']]

In [210]:
# Define the transformations for each column
preprocessor = ColumnTransformer(
transformers=[
    ('minmax_scaler',MinMaxScaler(),['BEDS','BATHS','SQUARE FEET','YEAR BUILT'])
])

# Apply the transformations in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

df_NN=pipeline.fit_transform(df_NN)

### Need to reduce impact of Year Built, so will probably require custom distance func

In [211]:
input_df = pd.DataFrame({'BEDS':[2],'BATHS':[2],'SQUARE FEET':[1800],'YEAR BUILT':[5000]})
input_df

input_t=pipeline.transform(input_df)
input_t


array([[2.02020202e-02, 3.00000000e-04, 8.64817406e-05, 2.28832952e+00]])

In [212]:
input_t[0][0]

0.020202020202020204

In [242]:
from scipy.spatial.distance import euclidean
from sklearn.neighbors import DistanceMetric

def weighted_euclidean(x,y):
    weights=np.array([1,1,1,0])
    
    return euclidean(x,y,weights)


# NEED TO FIGURE OUT HOW TO APPLY WEIGHTS PROPERLY


In [243]:

knn=NearestNeighbors(n_neighbors=df_NN.shape[0],metric=weighted_euclidean)
knn.fit(df_NN)

distances, indices = knn.kneighbors(input_t)

        
        

    

In [244]:
input_t[0]

array([2.02020202e-02, 3.00000000e-04, 8.64817406e-05, 2.28832952e+00])

In [245]:
df_NN[0]

array([4.04040404e-02, 3.00000000e-04, 9.49424334e-05, 5.82951945e-01])

In [246]:
input_t

array([[2.02020202e-02, 3.00000000e-04, 8.64817406e-05, 2.28832952e+00]])

In [247]:
def loss_function(index,input,output):
    lst=[]
    
    for i in range(0,3):
        loss=np.abs((input[0][i]-output[index][i]))
        
        lst.append(loss)
        
    return np.mean(lst)

In [248]:
for index in indices[0][0:5]:
    vals=[]
    val=loss_function(index,input_t,df_NN)
    vals.append(val)
    
val_avg=np.mean(vals)

In [241]:
val_avg

0.003368381434391086

In [249]:
val_avg

0.0

In [229]:
df_full.loc[indices[0][0]]

ADDRESS                      1103 Fitzgerald Ct
CITY                                Summerville
STATE OR PROVINCE                            SC
ZIP OR POSTAL CODE                        29485
PROPERTY TYPE         Single Family Residential
PRICE                                  503805.0
BEDS                                        5.0
BATHS                                       3.0
SQUARE FEET                              2226.0
YEAR BUILT                               2748.0
DAYS ON MARKET                             20.0
LATITUDE                                32.9431
LONGITUDE                              -80.2772
Name: 281733, dtype: object

In [278]:
all=df_full[(df_full['PROPERTY TYPE']=='Single Family Residential')]

state=df_full[(df_full['PROPERTY TYPE']=='Single Family Residential')&
               (df_full['STATE OR PROVINCE']=='TX')]

city=df_full[(df_full['PROPERTY TYPE']=='Single Family Residential')&
               (df_full['STATE OR PROVINCE']=='TX')&
               (df_full['CITY']=='Houston')]

zip=df_full[(df_full['PROPERTY TYPE']=='Single Family Residential')&
               (df_full['STATE OR PROVINCE']=='TX')&
               (df_full['CITY']=='Houston')&
               (df_full['ZIP OR POSTAL CODE']=='77004')]

slices=[all,state,city,zip]

In [279]:
def loss_function(index,input,output):
    lst=[]
    
    for i in range(0,3):
        loss=np.abs((input[0][i]-output[index][i]))
        
        lst.append(loss)
        
    return np.mean(lst)
        
    


In [280]:
# OPTIMUM WEIGHT FOR YEAR BUILT SEARCH

# Beware, could take a VERY long time

results={} # Dict of dicts of lists of tuples

for i,slice in enumerate(slices):

    str=""
            
    if i==0:
        str='all'
        
    elif i==1:
        str='state'
        
    elif i==2:
        str='city'
        
    else:
        str='zip'

    results[str]={}
    
    preprocessor = ColumnTransformer(
    transformers=[
        ('minmax_scaler',MinMaxScaler(),['BEDS','BATHS','SQUARE FEET'])
    ])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
    
    slice_np=pipeline.fit_transform(slice[['BEDS','BATHS','SQUARE FEET']])
    
    for year in [2000,2020,1900]:
        
        input_df = pd.DataFrame({'BEDS':[3],'BATHS':[3],'SQUARE FEET':[2600]})
        input_df

        input_t=pipeline.transform(input_df)
        input_t
        
        dct={"weights":[],"loss":[]}

        for w in np.linspace(0,1,21):
            
            def weighted_euclidean(x:np.array,y:np.array):
                weights=np.array([1,1,1,w])
                
                return euclidean(x,y,weights)
            
            knn=NearestNeighbors(n_neighbors=slice_np.shape[0],metric=weighted_euclidean)
            knn.fit(slice_np)

            distances, indices = knn.kneighbors(input_t)
            
            for index in indices[0][0:5]:
                vals=[]
                val=loss_function(index,input_t,slice_np)
                vals.append(val)
                
            val_avg=np.mean(vals)
                
            dct['weights'].append(w)
            dct['loss'].append(val_avg)
                
                
        results[str][year]=dct
            
        
                
                
            

            
        


In [284]:

results_df=pd.DataFrame(results['city'][1900])


    

In [285]:
results_df

,weights,loss
0,0.00,0.000652
1,0.05,0.018803
2,0.10,0.021078
3,0.15,0.021078
4,0.20,0.021078
5,0.25,0.020951
6,0.30,0.034936
7,0.35,0.038553
8,0.40,0.023034
9,0.45,0.023034



### Warning: Regarding the Nearest Neighbors algorithms,if two neighbors and have identical distances but different labels, the result will depend on the ordering of the training data.

In [108]:
df_slice=df_full[df_full['ZIP OR POSTAL CODE']=='78705']

In [109]:
df_slice

,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PROPERTY TYPE,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,DAYS ON MARKET,LATITUDE,LONGITUDE


In [110]:
picks=[]

for i in indices[0]:
    try:
        pick=df_slice.loc[i]
        picks.append(pick)
    except:
        pass
    
    if len(picks)==3:
        break

In [111]:
picks[0]

IndexError: list index out of range

In [91]:
picks[1]

ADDRESS                   2501 Truxillo St #6
CITY                                  Houston
STATE OR PROVINCE                          TX
ZIP OR POSTAL CODE                      77004
PROPERTY TYPE         Multi-Family (2-4 Unit)
PRICE                                780000.0
BEDS                                      9.0
BATHS                                     1.0
SQUARE FEET                            5000.0
YEAR BUILT                             1930.0
DAYS ON MARKET                           62.0
LATITUDE                             29.72961
LONGITUDE                          -95.367364
Name: 349536, dtype: object

In [71]:
df_full.iloc[349519]

ADDRESS                          3342 Reeves St
CITY                                    Houston
STATE OR PROVINCE                            TX
ZIP OR POSTAL CODE                        77004
PROPERTY TYPE         Single Family Residential
PRICE                                  250000.0
BEDS                                        3.0
BATHS                                       2.0
SQUARE FEET                              1104.0
YEAR BUILT                               2002.0
DAYS ON MARKET                             49.0
LATITUDE                              29.727252
LONGITUDE                            -95.355255
Name: 349519, dtype: object

In [298]:
input_t

array([[3.03030303e-02, 3.00000000e-04, 8.64817406e-05, 5.72082380e-01]])